In [9]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'Chapter2/wine.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
wine = pd.read_csv(file_content_stream)
wine.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [2]:
wine['Wine'].value_counts()

2    71
1    59
3    48
Name: Wine, dtype: int64

## Data Splitting

In [3]:
## Defining input and target variables
X = wine.drop(columns = 'Wine', axis = 1)
Y = wine['Wine']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Standardizing the data 
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## One-vs-all Classifier

In [4]:
## Building the multi-classifier (using RF) 
one_vs_all_RF = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test
one_vs_all_RF_pred = one_vs_all_RF.predict_proba(X_test)
one_vs_all_RF_pred = np.argmax(one_vs_all_RF_pred, axis = 1) + 1
one_vs_all_RF_pred

array([2, 2, 2, 1, 1, 1, 1, 3, 2, 2, 1, 3, 2, 3, 1, 3, 3, 1, 2, 1, 3, 2,
       2, 2, 3, 3, 3, 2, 2, 2, 3, 1, 1, 1, 1, 1])

In [5]:
## Building the multi-classifier (using SVM) 
one_vs_all_svm = OneVsRestClassifier(estimator = SVC(kernel = 'rbf', probability = True)).fit(X_train, Y_train)

## Predicting on the test
one_vs_all_svm_pred = one_vs_all_svm.predict_proba(X_test)
one_vs_all_svm_pred = np.argmax(one_vs_all_svm_pred, axis = 1) + 1
one_vs_all_svm_pred

array([2, 2, 2, 1, 1, 1, 1, 3, 2, 2, 1, 3, 2, 3, 1, 3, 3, 2, 2, 1, 3, 2,
       2, 2, 2, 3, 3, 2, 2, 2, 3, 1, 1, 1, 1, 1])

## One-vs-One Classifier

In [7]:
## Building the multi-classifier (using RF) 
one_vs_one_RF = OneVsOneClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test
one_vs_one_RF_pred = one_vs_one_RF.predict_proba(X_test)
one_vs_one_RF_pred

AttributeError: 'OneVsOneClassifier' object has no attribute 'predict_proba'

In [8]:
## Building the multi-classifier (using SVM) 
one_vs_one_svm = OneVsOneClassifier(estimator = SVC(kernel = 'rbf', probability = True)).fit(X_train, Y_train)

## Predicting on the test
one_vs_one_svm_pred = one_vs_all_svm.predict_proba(X_test)
# one_vs_all_svm_pred = np.argmax(one_vs_one_svm_pred, axis = 1) + 1
one_vs_one_svm_pred

array([[0.01717633, 0.97841162, 0.00441205],
       [0.19496436, 0.80155667, 0.00347897],
       [0.04967798, 0.94388298, 0.00643904],
       [0.68549951, 0.29870337, 0.01579713],
       [0.92311496, 0.05256429, 0.02432075],
       [0.66554654, 0.30072915, 0.03372431],
       [0.89808901, 0.07921843, 0.02269257],
       [0.00947272, 0.00936942, 0.98115786],
       [0.04073457, 0.87725704, 0.0820084 ],
       [0.00382624, 0.98441993, 0.01175383],
       [0.97802631, 0.01267222, 0.00930147],
       [0.01533878, 0.27352307, 0.71113815],
       [0.00439253, 0.99330562, 0.00230185],
       [0.00894886, 0.27252332, 0.71852782],
       [0.80152108, 0.18626539, 0.01221352],
       [0.00830024, 0.07086506, 0.9208347 ],
       [0.01214352, 0.01602494, 0.97183154],
       [0.11234383, 0.84687116, 0.04078501],
       [0.00481962, 0.99125224, 0.00392815],
       [0.97687497, 0.00659205, 0.01653298],
       [0.00930433, 0.03432541, 0.95637025],
       [0.00482692, 0.99229843, 0.00287465],
       [0.

In [ ]:
## Building the multi-classifier (using SVM) 
one_vs_one_svm = OneVsOneClassifier(estimator = SVC(kernel = 'rbf', probability = True)).fit(X_train, Y_train)

## Predicting on the test
one_vs_one_svm_pred = one_vs_all_svm.predict_proba(X_test)
